In [1]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

le = LabelEncoder()

df = pd.read_csv('../data/train_data_fix_concat.csv')

df['oil_type'] = df['oil_type'].fillna(value='unknown')
df['oil_property_param_value'] = df['oil_property_param_value'].fillna(value=-1)
df['component_class'] = df['component_class'].fillna(value='unknown')
df['component_property_param_value'] = df['component_property_param_value'].fillna(value=-1)
df['smiles'] = df['smiles'].fillna(value='unknown')
df['polymer'] = df['polymer'].fillna(value='unknown')
df = df.drop(columns=['smiles', 'polymer', 'component_class'])

blend_ids_with_target = df[df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8']['blend_id'].unique().tolist()
df = df[df['blend_id'].isin(blend_ids_with_target)].reset_index(drop=True)

target_df = df[df['oil_property_param_title'] == 'ad7e6027-00b8-4c27-918c-d1561f949ad8'][['blend_id', 'oil_property_param_value']].drop_duplicates()
df = df[df['oil_property_param_title'] != 'ad7e6027-00b8-4c27-918c-d1561f949ad8']
df['oil_type'] = le.fit_transform(df['oil_type'])
cols=df.columns
from sklearn.preprocessing import MinMaxScaler
mms=MinMaxScaler()
newdf=mms.fit_transform(df.select_dtypes("number"))

In [2]:
df[df.select_dtypes("number").columns]=newdf

In [5]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from spektral.data import Graph



In [6]:
d=df.set_index(["blend_id","oil_property_param_title","component_name","component_property_param_title"])

In [7]:
d

oil_type  \
blend_id                             oil_property_param_title             component_name                       component_property_param_title                   
49743a76-a614-11ee-9529-005056921581 4c7a51f1-dc82-41dc-92fa-772535c2c70c 615537f6-1f8f-4240-a5e9-8f7be344ecd3 02236ee6-5eec-4368-a2e4-6f2e73fb0f96      0.00   
                                                                                                               2511714c-ab50-4566-bc92-8e4095d87d01      0.00   
                                                                                                               38b06d9a-bcf2-40de-8e1b-9c3988c42406      0.00   
                                                                                                               f216a0c7-d453-4b15-9b4a-7647cbe2d874      0.00   
                                                                                                               9703e283-f529-4fdb-8d84-24413b2b4338      0.00   
...                                                                                                                                                       ...   
45f1e44a-9410-11ee-8abf-005056921581 4ff1ad46-aa06-4344-875e-a1f33ac094a7 1433126f-b36a-4e69-ad2f-ee12d8b46275 a59e9688-c5e5-4488-a911-2fc21e0355be      0.25   
                                                                                                               f400be8b-3bed-4f17-8a12-9fabd9b73586      0.25   
                                                                                                               1931ea10-9783-4065-a77c-04532d7d0559      0.25   
                                                                                                               c11112c4-487f-4e2c-812d-84a29b240b69      0.25   
                                                                                                               c11112c4-487f-4e2c-812d-84a29b240b69      0.25   

                                                                                                                                                     oil_property_param_value  \
blend_id                             oil_property_param_title             component_name                       component_property_param_title                                   
49743a76-a614-11ee-9529-005056921581 4c7a51f1-dc82-41dc-92fa-772535c2c70c 615537f6-1f8f-4240-a5e9-8f7be344ecd3 02236ee6-5eec-4368-a2e4-6f2e73fb0f96                  0.000463   
                                                                                                               2511714c-ab50-4566-bc92-8e4095d87d01                  0.000463   
                                                                                                               38b06d9a-bcf2-40de-8e1b-9c3988c42406                  0.000463   
                                                                                                               f216a0c7-d453-4b15-9b4a-7647cbe2d874                  0.000463   
                                                                                                               9703e283-f529-4fdb-8d84-24413b2b4338                  0.000463   
...                                                                                                                                                                       ...   
45f1e44a-9410-11ee-8abf-005056921581 4ff1ad46-aa06-4344-875e-a1f33ac094a7 1433126f-b36a-4e69-ad2f-ee12d8b46275 a59e9688-c5e5-4488-a911-2fc21e0355be                  0.000514   
                                                                                                               f400be8b-3bed-4f17-8a12-9fabd9b73586                  0.000514   
                                                                                                               1931ea10-9783-4065-a77c-04532d7d0559                  0.000514   
                                                                                                            

In [48]:
import numpy as np
from spektral.data import Graph

def make_graphs(df, targets):
    # Установите blend_id как индекс, если это еще не сделано
    if df.index.name != 'blend_id':
        df = df.set_index('blend_id')

    graphs = []
    for blend_id in df.index.unique():
        graph = make_graph(df, targets, blend_id)
        graphs.append(graph)
    print("Графы готовы")
    return graphs

def make_graph(df, targets, blend_id):
    df_take = df.loc[blend_id]
    y = np.array([targets.query("blend_id == @blend_id").iloc[0, 1]], dtype=np.float32)
    if np.isnan(y).any():
        print("Отсутствует целевое значение для:", blend_id)

    x, edge_index = [], []
    blend(df_take.copy(), x, edge_index)
    x = np.array(x, dtype=np.float32).reshape(-1, 1)  # Убедитесь, что x имеет форму (num_nodes, num_features)
    edge_index = np.array(edge_index, dtype=np.int32).T  # (2, num_edges)

    graph = Graph(x=x, a=edge_index, y=y)
    return graph

def blend(df, x, edge_index):
    index_map = {'blend': 0}
    x.append(0)  # Добавление свойства для узла смеси

    # Обход свойств нефти
    for oil_prop_title, row in df.groupby('oil_property_param_title'):
        idx = oil_prop(row, x, edge_index, index_map)
        index_map[oil_prop_title] = idx
        edge_index.append([0, idx, 1])  # Добавление ребра от blend к свойству

def oil_prop(df, x, edge_index, index_map):
    val = df['oil_property_param_value'].iloc[0]
    x.append(val)
    my_idx = len(x) - 1

    # Обход компонентов
    for comp, row in df.groupby('component_name'):
        idx = component(row, x, edge_index, index_map)
        index_map[comp] = idx
        edge_index.append([my_idx, idx, 1])  # Ребро от свойства нефти к компоненту

    return my_idx

def component(df, x, edge_index, index_map):
    x.append(0)  # Добавление пустого значения для узла компонента
    my_idx = len(x) - 1

    # Обход свойств компонента
    for comp_prop, row in df.groupby('component_property_param_title'):
        idx, val = component_prop(row, x, edge_index, index_map)
        index_map[comp_prop] = idx
        edge_index.append([my_idx, idx, val])

    return my_idx

def component_prop(df, x, edge_index, index_map):
    val = df['component_property_param_value'].iloc[0]
    x.append(0)  # Добавление пустого узла для свойства
    my_idx = len(x) - 1
    return my_idx, val


In [49]:
graphs=make_graphs(df,target_df)

Графы готовы


In [53]:
len(graphs[0])

3

In [28]:
# def GraphCreate(df,target):
#     # Create edge index and edge attributes
#     df.index=range(0,df.shape[0])
#     edge_index = []
#     edge_attr = []
    
#     for _, row in df.iterrows():
#         blend_id = row['blend_id']
        
        
#         oil_prop_title = row['oil_property_param_title']
#         component_name = row['component_name']
#         comp_prop_name = row['component_property_param_title']

#         blend_node_index = df[df['blend_id'] == blend_id].index[0]
#         oil_prop_node_index = df[df['oil_property_param_title'] == oil_prop_title].index[0]
#         component_node_index = df[df['component_name'] == component_name].index[0]
#         comp_prop_node_index = df[df['component_property_param_title'] == comp_prop_name].index[0]

#         edge_index.append([blend_node_index, oil_prop_node_index,row['oil_property_param_value']])
#         edge_index.append([oil_prop_node_index, comp_prop_node_index,row['component_property_param_value']])
#         edge_index.append([comp_prop_node_index, component_node_index,row['component_property_param_value']])

#         edge_attr.append(row['oil_property_param_value'])
#         edge_attr.append(row['component_property_param_value'])
#         edge_attr.append(row['component_property_param_value'])

#     # Convert to NumPy arrays
#     edge_index = np.array(edge_index)
#     x = df.index.values.reshape(-1, 1)  # Node features
#     edge_attr = np.array(edge_attr).reshape(-1, 1)  # Edge features
#     print(edge_index.shape,edge_attr.shape)
#     print(target)
#     graph = Graph(x=np.arange(0,edge_index.shape[0]), a=edge_index, y=target)

#     return graph

In [29]:
import joblib

joblib.dump(graphs,"train_graphs.pkl")
graphs=joblib.load("train_graphs.pkl")

In [30]:
graphs[:300]

[Graph(n_nodes=337, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=331, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=243, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=421, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=1531, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=3382, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=421, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=5147, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=505, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=1327, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=977, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=606, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=331, n_node_features=1, n_edge_features=None, n_labels=1),
 Graph(n_nodes=491, n_node_feature

In [31]:
from spektral.data import Dataset

class MyGraphDataset(Dataset):
    def __init__(self, graph_list, **kwargs):
        self.graph_list = graph_list  # Список объектов Graph
        super().__init__(**kwargs)

    def read(self):
        return self.graph_list
    def __len__(self):
        return len(self.graphs)

    def __getitem__(self, idx):
        return self.graphs[idx]

In [54]:
train_ds=MyGraphDataset(graphs)
# val_ds=MyGraphDataset(graphs[380:])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [33]:
train_ds[0]

Graph(n_nodes=337, n_node_features=1, n_edge_features=None, n_labels=1)

In [38]:
import spektral
from tensorflow.keras.models import Model
from spektral.layers import GCNConv,GlobalAvgPool,GATConv
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
class GraphModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(128, activation='relu')
        self.gat=GATConv(256,activation='relu')
        self.pool = GlobalAvgPool()  # Global pooling to aggregate information across the graph
        self.dropout = Dropout(0.5)  # Dropout for regularization
        self.hidden=Dense(128,activation="relu")
        self.hidden1=Dense(64,activation="relu")
        self.hidden2=Dense(32,activation="relu")
        self.dense = Dense(1, activation=None)  # Output layer for regression

    def call(self, inputs):
        print(inputs)
        x, a = inputs
        x = tf.cast(x, tf.float32)  # Ensure x is of type float32
        x = self.conv1([x, a])  # First GCN convolution layer  # Second GCN convolution layer
        x= self.gat([x,a])
        x = self.pool(x)  # Apply global average pooling
        x = self.dropout(x)  # Apply dropout
        x=self.hidden(x)
        x=self.hidden1(x)
        x=self.hidden2(x)
        return self.dense(x)  # Output layer
model=GraphModel()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])

In [39]:
from spektral.data import BatchLoader,Loader
loader = BatchLoader(train_ds, batch_size=1, epochs=30)
#val_loader = BatchLoader(val_ds, batch_size=1, epochs=30)

In [40]:
model.fit(train_ds[0],epochs=30,shuffle=True)

ValueError: Unrecognized data type: x=Graph(n_nodes=337, n_node_features=1, n_edge_features=None, n_labels=1) (of type <class 'spektral.data.graph.Graph'>)

In [36]:
import spektral
from spektral.data import Loader, Graph
from spektral.models import GCN

# Предполагая, что train_ds является списком или DataSet объектами Graph
loader = Loader(train_ds)  # Используем Loader из Spektral для загрузки данных

model = GCN(n_labels=1)  # Создание модели GNN, предназначенной для графов
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])
# Тренировка модели
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=30, shuffle=True)

ValueError: Unrecognized data type: x=<spektral.data.loaders.Loader object at 0x000001C44E59B390> (of type <class 'spektral.data.loaders.Loader'>)

In [44]:
import numpy as np
import spektral
from spektral.data import Graph, Loader
from spektral.layers import GCNConv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Создаем пример данных
graphs = [Graph(x=np.random.rand(1, 10), a=np.array([[0]]), y=np.random.rand(1)) for _ in range(10)]

# Оборачиваем графы в DataLoader
loader = Loader(graphs, batch_size=1, epochs=30)

# Построение модели GCN для регрессии
class GCNRegression(Model):
    def __init__(self, n_input_channels, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = GCNConv(32, activation='relu')
        self.conv2 = GCNConv(16, activation='relu')
        self.dense = Dense(1)  # Один выходной нейрон для регрессии

    def call(self, inputs):
        x, a = inputs
        x = self.conv1([x, a])
        x = Dropout(0.5)(x)  # Dropout для регуляризации
        x = self.conv2([x, a])
        x = Dropout(0.5)(x)
        x = self.dense(x)
        return x

# Инициализация модели
model = GCNRegression(n_input_channels=10)
model.compile(optimizer=Adam(learning_rate=0.01), loss=MeanSquaredError(), metrics=['mean_squared_error'])

# Тренировка модели
model.fit(loader, epochs=30, shuffle=True)


ValueError: Unrecognized data type: x=<spektral.data.loaders.Loader object at 0x000001C44E357AD0> (of type <class 'spektral.data.loaders.Loader'>)

In [ ]:
import spektral
from spektral.models import GeneralGNN
from spektral.data import Graph

# Создаем модель графовой нейронной сети.
model=GraphModel()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error', metrics=['mean_absolute_error'])

# Обучаем модель, используя BatchLoader для загрузки данных.
loader = spektral.data.BatchLoader(train_ds, batch_size=32)
# your_data - это объект Dataset, содержащий ваши графы.
epochs=20
# Цикл обучения.
model.fit(loader.load(), steps_per_epoch=loader.steps_per_epoch, epochs=epochs)

In [ ]:
import numpy as np

# Assuming `model` is your Keras model
# `loader` is your Spektral data loader for training data
# `val_loader` is your Spektral data loader for validation data

# Custom training loop
for epoch in range(30):  # Number of epochs
    print("Epoch:", epoch + 1)
    for batch in loader:  # Train on batches
        inputs, targets = batch
        model.train_on_batch(inputs, targets)
    
    # Validation
    val_loss, val_acc = [], []
    for batch in val_loader:  # Validate on batches
        inputs, targets = batch
        loss, acc = model.test_on_batch(inputs, targets)
        val_loss.append(loss)
        val_acc.append(acc)
    
    # Print validation results
    print("Validation Loss:", np.mean(val_loss), "Validation Accuracy:", np.mean(val_acc))

# Ensure you handle exceptions and potentially use `try-except` blocks to catch and handle errors.


In [ ]:

from keras.layers import LSTM
from keras.layers import Dense
class RNN(Model):
    def __init__(self):
        super().__init__()
        self.lstm = LSTM(128,input_shape=(1,None,3),activation="tanh")
        self.dense = Dense(1, activation=None)

    def call(self, inputs):
        x, a = inputs
        print(inputs)
        return self.dense(self.lstm)  # Output layer
rnn_model=RNN()
print('a')
rnn_model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
rnn_model.fit(loader.load(),epochs=30,shuffle=True,validation_data=val_loader.load())

In [ ]:
ragged_tensor = tf.ragged.constant([g.a for g in graphs])

In [ ]:
y=[g.y for g in graphs]
y_array = np.array(y, dtype=np.float32)  # Convert to NumPy array
y_tensor = tf.convert_to_tensor(y_array, dtype=tf.float32)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
rnn_m=Sequential([
    LSTM(32,input_shape=(None,3),activation="tanh",return_sequences=False),
    Dense(1, activation=None)
    
])
rnn_m.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error', metrics=['mean_absolute_error'])

In [ ]:
rnn_m.fit(ragged_tensor,y_tensor)

In [ ]:
print("Input shape:", ragged_tensor.shape)
print("Target shape:", y_array.shape)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available.")
else:
    print("GPU is not available.")